In [57]:
import json
import numpy as np
import pandas as pd
from boot_util import *
import os
import json

import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [58]:
def get_out_dir_path(conf):
    return f"{conf['out_dir']}/{conf['sample_type']}_n_{conf['n_train']}_p_{conf['patience']}_s_{conf['seed']}_c_{conf['control']}_p_{conf['m']}_d_{conf['depth']}_w_{conf['layer_size']}"

In [59]:
confs_path = ["conf_1.json","conf_2.json","conf_4.json"]

In [60]:
def transform_log(S):
    #return np.log(1-S)
    S = np.clip(S,0,1)
    return np.arcsin((1-S)**0.5)

def inverse_transform_log(U):
    U = np.clip(U,0,transform_log(0))
    #return -1*(np.exp(U)) + 1
    return -1*(np.sin(-1*U)**2) + 1
    #return np.sin(U)**2

In [63]:
save_d_95 = []
save_d_90 = []
save_d_95_naive = []
save_d_90_naive = []
anls = []
for conf_path in confs_path:
    confs = json.load(open(conf_path,"r"))
    for conf in confs:
        bdir = get_out_dir_path(conf)
        if conf["sample_type"] == "LPH":
            sm = SimStudyLinearPH()
        elif conf["sample_type"] == "NLPH":
            sm = SimStudyNonLinearPH()
        elif conf["sample_type"] == "NLNPH":
            sm = SimStudyNonLinearNonPH_smooth()
        elif conf["sample_type"] == "Deep1":
            sm = SimStudyDeep1()
        elif conf["sample_type"] == "Deep2":
            sm = SimStudyDeep2()
        else:
            raise Exception("Unkowne sample method")
        if "analasys_offset" in conf.keys():
            offset_start,offset_end = conf["analasys_offset"]
        preds_all = pd.read_csv(f"{bdir}/res_boot_samp_0_boot_0.csv",compression="gzip") 
        preds_all = preds_all.drop(list(preds_all)[0],axis=1).set_index("t").drop(0.0)
        df_test = pd.read_csv(f"{bdir}/test.csv",compression="gzip")

        grid_start, grid_end, grid_step = conf["grid"]
        grid = np.arange(grid_start,grid_end,grid_step)[1:]
        x = np.array(df_test[conf["cols_leave"]])
        df_test = pd.DataFrame([np.exp(-1*sm.cum_hazard(t,x)) for t in grid]).set_index(preds_all.index)

        if "analasys_offset" in conf.keys():
            df_test =df_test.iloc[offset_start:offset_end]
        
        count_90 = np.zeros(df_test.shape[1])
        count_95 = np.zeros(df_test.shape[1])
        
        count_90_W = np.zeros(df_test.shape[1])
        count_95_W = np.zeros(df_test.shape[1])

        count_90_naive = np.zeros(df_test.shape[1])
        count_95_naive = np.zeros(df_test.shape[1])
        thetha_bias = 0
        m_bias = 0
        avg_bw_len_90 = 0
        avg_bw_len_95 = 0

        avg_bw_len_90_naive = 0
        avg_bw_len_95_naive = 0
        
        avg_bw_len_90_W = 0
        avg_bw_len_95_W = 0
        for j in range(conf["n_samp"]):
            print(j)
            preds_all = pd.read_csv(f"{bdir}/res_boot_samp_{j}_boot_0.csv",compression="gzip") 
            preds_all = preds_all.drop(list(preds_all)[0],axis=1).set_index("t").drop(0.0)
            if "analasys_offset" in conf.keys():
                preds_all = preds_all.iloc[offset_start:offset_end]
            
            thetha = pd.read_csv(f"{bdir}/theta_{j}.csv",compression="gzip") 
            thetha = thetha.drop(list(thetha)[0],axis=1).set_index("t").drop(0.0)        
            if "analasys_offset" in conf.keys():
                thetha = thetha.iloc[offset_start:offset_end]
            
            distances = []
            distances_naive = []
            distances_by_t = []
            distances_W = []
            for i in range(1,conf["n_boot"]):
                preds_boot = pd.read_csv(f"{bdir}/res_boot_samp_{j}_boot_{i}.csv",compression="gzip") 
                preds_boot = preds_boot.drop(list(preds_boot)[0],axis=1).set_index("t").drop(0.0)
                if "analasys_offset" in conf.keys():
                    preds_boot = preds_boot.iloc[offset_start:offset_end]
                
                distances += [pd.DataFrame(np.array(preds_boot) - np.array(preds_all)).abs().max(axis=0)]
                distances_by_t += [pd.DataFrame(np.array(preds_boot) - np.array(preds_all))]
                distances_naive += [pd.DataFrame(np.array(preds_boot) - np.array(thetha)).abs().max(axis=0)]
                #distances_W += [((transform_log(preds_boot) - transform_log(preds_all))).abs().max(axis=0)]
                clp = np.clip(preds_boot,0.01,0.99)
                W = (clp*(1-clp))**0.5
                distances_W += [((preds_boot - preds_all)/W).abs().max(axis=0)]
            bias = []
            for pos in range(df_test.shape[1]):
                aa = np.stack([x[pos] for x in distances_by_t])
                bias += [aa.mean(axis=0)]
            bias = np.stack(bias)
            distances = pd.concat(distances,axis=1)
            distances_naive = pd.concat(distances_naive,axis=1)
            d_90 = np.percentile(distances,90,axis=1)
            d_90_naive = np.percentile(distances_naive,90,axis=1)
            d_95 = np.percentile(distances,95,axis=1)
            d_95_naive = np.percentile(distances_naive,95,axis=1)
            distances_W = pd.concat(distances_W,axis=1)
            d_95_W = np.percentile(distances_W,95,axis=1)
            d_90_W = np.percentile(distances_W,90,axis=1)

            save_d_95 += [d_95]
            save_d_90 += [d_90]
            save_d_95_naive += [d_95_naive]
            save_d_90_naive += [d_90_naive]
            l = 0
            for idx, row in df_test.T.iterrows():
                res_real = np.array(row)
                res_all = thetha[str(idx)]
                m_all = preds_all[str(idx)]
                bw_90 = d_90[l]
                bw_90_naive = d_90_naive[l]
                bw_95 = d_95[l]
                bw_95_naive = d_95_naive[l]

                bias_i = bias[l,:] 

                in_bw_90 = ((res_real > (res_all + bw_90)).sum() + (res_real < (res_all - bw_90)).sum() > 0)
                in_bw_95 = ((res_real > (res_all + bw_95)).sum() + (res_real < (res_all - bw_95)).sum() > 0)
                count_90[l] += in_bw_90
                count_95[l] += in_bw_95
                
                
                clp = np.clip(res_all,0.01,0.99)
                W = (clp*(1-clp))**0.5
                
                
                bw_W = d_90_W[l]
                in_bw_90_W = ((res_real > (res_all + bw_W*W)).sum() + (res_real < (res_all - bw_W*W)).sum() > 0)
                avg_bw_len_90_W += bw_W*W.mean()
                
                bw_W = d_95_W[l]
                in_bw_95_W = ((res_real > (res_all + bw_W*W)).sum() + (res_real < (res_all - bw_W*W)).sum() > 0)
                avg_bw_len_95_W += bw_W*W.mean()
                #in_bw_90_W = ((res_real < ((inverse_transform_log(transform_log(res_all) + d_90_W[i])))).sum() + (res_real > ((inverse_transform_log(transform_log(res_all) - d_90_W[i])))).sum() > 0)
                #in_bw_95_W = ((res_real < ((inverse_transform_log(transform_log(res_all) + d_95_W[i])))).sum() + (res_real > ((inverse_transform_log(transform_log(res_all) - d_95_W[i])))).sum() > 0)
                count_90_W[l] += in_bw_90_W
                count_95_W[l] += in_bw_95_W

                in_bw_90_naive = ((res_real > (res_all + bw_90_naive)).sum() + (res_real < (res_all - bw_90_naive)).sum() > 0)
                in_bw_95_naive = ((res_real > (res_all + bw_95_naive)).sum() + (res_real < (res_all - bw_95_naive)).sum() > 0)
                count_90_naive[l] += in_bw_90_naive
                count_95_naive[l] += in_bw_95_naive
                
                #avg_bw_len_90_W += np.abs(inverse_transform_log(transform_log(res_real) + d_90_W[i]) - inverse_transform_log(transform_log(res_real) - d_90_W[i])).mean()
                #avg_bw_len_95_W += np.abs(inverse_transform_log(transform_log(res_real) + d_95_W[i]) - inverse_transform_log(transform_log(res_real) - d_95_W[i])).mean()

                
                thetha_bias += (res_real - res_all).abs().max()
                m_bias += (res_real - m_all).abs().max()
                avg_bw_len_90 += bw_90
                avg_bw_len_95 += bw_95

                avg_bw_len_90_naive += bw_90_naive
                avg_bw_len_95_naive += bw_95_naive
                l+=1
                


        anls += [
            {
            "bdir" : bdir,
            "cover_90" : (1 - (count_90/conf["n_samp"]).mean()),
            "cover_95" : (1 - (count_95/conf["n_samp"]).mean()),
            "cover_90_W" : (1 - (count_90_W/conf["n_samp"]).mean()),
            "cover_95_W" : (1 - (count_95_W/conf["n_samp"]).mean()),
            "cover_90_naive" : (1 - (count_90_naive/conf["n_samp"]).mean()),
            "cover_95_naive" : (1 - (count_95_naive/conf["n_samp"]).mean()),
            "bias_thetha" : thetha_bias /(conf["n_samp"]*df_test.shape[1]),
            "bias_m" : m_bias /(conf["n_samp"]*df_test.shape[1]),
            "bw_90" : avg_bw_len_90 /(conf["n_samp"]*df_test.shape[1]),
            "bw_95" : avg_bw_len_95 /(conf["n_samp"]*df_test.shape[1]),
            "bw_90_W" : avg_bw_len_90_W /(conf["n_samp"]*df_test.shape[1]),
            "bw_95_W" : avg_bw_len_95_W /(conf["n_samp"]*df_test.shape[1]),
            "bw_90_naive" : avg_bw_len_90_naive /(conf["n_samp"]*df_test.shape[1]),
            "bw_95_naive" : avg_bw_len_95_naive /(conf["n_samp"]*df_test.shape[1]),
            }
        ]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [65]:
pd.DataFrame(anls)

,bdir,cover_90,cover_95,cover_90_W,cover_95_W,cover_90_naive,cover_95_naive,bias_thetha,bias_m,bw_90,bw_95,bw_90_W,bw_95_W,bw_90_naive,bw_95_naive
0,../sims//NLNPH_n_10000_p_25_s_1_c_1_p_100_d_6_...,0.90316,0.95102,0.88137,0.93262,0.98937,0.99692,0.075976,0.049317,0.130058,0.151398,0.091193,0.108586,0.157855,0.180131
1,../sims//NLNPH_n_10000_p_25_s_1_c_2_p_100_d_6_...,0.89062,0.94381,0.87092,0.92696,0.98508,0.99591,0.071182,0.048053,0.118033,0.137068,0.082655,0.097348,0.142785,0.162860
2,../sims//NLNPH_n_10000_p_25_s_1_c_4_p_100_d_6_...,0.89233,0.94460,0.87403,0.93126,0.97810,0.99305,0.066654,0.047628,0.110030,0.127595,0.077125,0.090207,0.131316,0.149804


In [66]:
for _,row in pd.DataFrame(anls).iterrows():
    print(f"{row.bdir} ---- & {row.cover_90_naive:.3f} & {row.bw_90_naive:.3f} & {row.cover_95_naive:.3f} & {row.bw_95_naive:.3f} & {row.bias_thetha:.3f} & {row.bias_m:.3f} & {row.cover_90:.3f} & {row.bw_90:.3f} & {row.cover_95:.3f} & {row.bw_95:.3f} & {row.cover_90_W:.3f} & {row.bw_90_W:.3f} & {row.cover_95_W:.3f} & {row.bw_95_W:.3f}")

../sims//NLNPH_n_10000_p_25_s_1_c_1_p_100_d_6_w_128 ---- & 0.989 & 0.158 & 0.997 & 0.180 & 0.076 & 0.049 & 0.903 & 0.130 & 0.951 & 0.151 & 0.881 & 0.091 & 0.933 & 0.109
../sims//NLNPH_n_10000_p_25_s_1_c_2_p_100_d_6_w_128 ---- & 0.985 & 0.143 & 0.996 & 0.163 & 0.071 & 0.048 & 0.891 & 0.118 & 0.944 & 0.137 & 0.871 & 0.083 & 0.927 & 0.097
../sims//NLNPH_n_10000_p_25_s_1_c_4_p_100_d_6_w_128 ---- & 0.978 & 0.131 & 0.993 & 0.150 & 0.067 & 0.048 & 0.892 & 0.110 & 0.945 & 0.128 & 0.874 & 0.077 & 0.931 & 0.090
